In [ ]:
#manipulação dos dados
import pandas as pd
url_dados = 'https://github.com/jhonatasisraelcl/Data_Science_with_python/blob/main/tweets_Pfizer/vaccination_tweets.csv?raw=true'
df = pd.read_csv(url_dados)

import  missingno

In [ ]:
df.head()

In [ ]:
df.shape
#mostra o tamanho do dataframe

In [ ]:
df.info()
#dá todas informações do dataframe
#objeto é um misto de numeros e textos

In [ ]:
df.describe()

# Avaliar dados faltantes

In [ ]:
df.isnull().any()

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
missingno.matrix(df, figsize=(40,10))

In [ ]:
df[df.isnull().any(axis=1)].head()

In [ ]:
linhas, colunas = df.shape
df.shape

In [ ]:
print(linhas)

In [ ]:
percentual_faltantes = (df.isnull().sum()/linhas) * 100

In [ ]:
print(percentual_faltantes)

In [ ]:
df.user_created